In [1]:
import duckdb
import yaml
import datetime
import openai

In [2]:
with open("config.yaml", "r") as stream:
    try:
        PARAM = yaml.safe_load(stream)
    except yaml.YAMLError as exc:
        print(exc)

openai.api_key  = PARAM['openai_api']
client = openai.OpenAI(api_key = PARAM['openai_api'])

def get_embedding(text, model=PARAM['vector_embedding_model']):
   text = text.replace("\n", " ")
   return client.embeddings.create(input = [text], model=model).data[0].embedding

In [4]:
con = duckdb.connect('md:')


Attempting to automatically open the SSO authorization page in your default browser.
Please open this link to login into your account: https://auth.motherduck.com/activate?user_code=SPFC-GPTG


Token successfully retrieved ✅

You can display the token and store it as an environment variable to avoid having to log in again:
  PRAGMA PRINT_MD_TOKEN;


In [ ]:
con.execute("install duckpgq from community;")
con.execute("load duckpgq;")

con.execute("install ducklake;")
con.execute("load ducklake;")

con.execute("install httpfs;")
con.execute("load httpfs;")

con.execute("install postgres;")
con.execute("load postgres;")

In [7]:
con.sql("SHOW DATABASES")

┌───────────────────────┐
│     database_name     │
│        varchar        │
├───────────────────────┤
│ drugdb                │
│ md_information_schema │
│ my_db                 │
│ sample_data           │
└───────────────────────┘

In [12]:
con.sql("""CREATE DATABASE my_ducklake (
    TYPE DUCKLAKE,
    DATA_INLINING_ROW_LIMIT 100
);""")

In [13]:
con.sql("SHOW DATABASES")

┌───────────────────────┐
│     database_name     │
│        varchar        │
├───────────────────────┤
│ drugdb                │
│ md_information_schema │
│ my_db                 │
│ my_ducklake           │
│ sample_data           │
└───────────────────────┘

In [14]:
con.sql(f"""
USE my_ducklake;
""")

In [15]:
command = """
CREATE TABLE Drug AS
    SELECT *
    FROM read_json_auto('drugs/drugs_8.json');
"""

con.execute(command)

In [16]:
command = """
CREATE TABLE Disease AS
    SELECT *
    FROM read_json_auto('drugs/disease_8.json');
"""

con.execute(command)

In [17]:
command = """
CREATE TABLE MOA AS
    SELECT *
    FROM 'drugs/moa_8.tsv';
"""

con.execute(command)

In [18]:
command = """
CREATE TABLE Drug_Disease AS
    SELECT *
    FROM 'drugs/cui_indication_8.tsv';
"""

con.execute(command)

In [19]:
command = """
CREATE TABLE Drug_MOA AS
    SELECT *
    FROM 'drugs/cui_moa_8.tsv';
"""

con.execute(command)

In [20]:
command = """
CREATE TABLE Trial AS
    SELECT *
    FROM 'drugs/trials_umls.tsv';
"""

con.execute(command)

In [21]:
con.sql(f"""
SELECT * FROM snapshots();
""")

┌─────────────┬───────────────────────────────┬────────────────┬────────────────────────────────────────────────────────────────┬─────────┬────────────────┬───────────────────┐
│ snapshot_id │         snapshot_time         │ schema_version │                            changes                             │ author  │ commit_message │ commit_extra_info │
│    int64    │   timestamp with time zone    │     int64      │                    map(varchar, varchar[])                     │ varchar │    varchar     │      varchar      │
├─────────────┼───────────────────────────────┼────────────────┼────────────────────────────────────────────────────────────────┼─────────┼────────────────┼───────────────────┤
│           0 │ 2026-01-07 10:14:52.213018+09 │              0 │ {schemas_created=[main]}                                       │ NULL    │ NULL           │ NULL              │
│           1 │ 2026-01-07 10:33:07.072785+09 │              1 │ {tables_created=[main.Drug], tables_inserted_into=

In [22]:
con.sql("""
CREATE PROPERTY GRAPH drug_graph
  VERTEX TABLES (
    Drug, Disease, MOA
  )
EDGE TABLES (
  Drug_Disease 	SOURCE KEY ('from') REFERENCES Drug (cui)
                DESTINATION KEY ('to') REFERENCES Disease (cui)
  LABEL MAY_TREAT,
  Drug_MOA SOURCE KEY ('from') REFERENCES Drug (cui)
          DESTINATION KEY ('to') REFERENCES MOA (id)
  LABEL HAS_MOA
);
""")

┌─────────┐
│ Success │
│ boolean │
├─────────┤
│ 0 rows  │
└─────────┘

In [23]:
con.sql("""SELECT DISTINCT disease_name
  FROM GRAPH_TABLE (drug_graph
  MATCH
  (mo:MOA WHERE mo.name = 'GABA B Receptor Interactions')<-[h:HAS_MOA]-(i:Drug)-[m:MAY_TREAT]->(c:Disease)
  COLUMNS (c.name AS disease_name)
  )
  LIMIT 5;
""")

┌─────────────────────┐
│    disease_name     │
│       varchar       │
├─────────────────────┤
│ Alzheimer's Disease │
│ Bipolar Disorder    │
│ Epilepsy            │
│ Mood Disorders      │
│ Absence Epilepsy    │
└─────────────────────┘

In [25]:
vector = get_embedding("joint-related pain")

print (vector[:2])

con.sql(f"""
SELECT cui, name
FROM Disease
ORDER BY array_cosine_similarity(definitionEmbedding::FLOAT[1536], {vector}::FLOAT[1536]) DESC
LIMIT 10;
""")

[-0.04786347970366478, 0.0007779016741551459]


┌──────────┬────────────────────────┐
│   cui    │          name          │
│ varchar  │        varchar         │
├──────────┼────────────────────────┤
│ C0038013 │ Ankylosing spondylitis │
│ C0022408 │ Arthropathy            │
│ C0004604 │ Back Pain              │
│ C0026858 │ Musculoskeletal Pain   │
│ C0949691 │ Spondylarthropathies   │
│ C0024031 │ Low Back Pain          │
│ C0040460 │ Toothache              │
│ C0003864 │ Arthritis              │
│ C0015468 │ Facial Pain            │
│ C0009088 │ Cluster Headache       │
├──────────┴────────────────────────┤
│ 10 rows                 2 columns │
└───────────────────────────────────┘

In [26]:
con.sql(f"""
  SELECT *
  FROM Trial
  WHERE "Study Title" LIKE '%MAXCLARITY%';
""")

┌────────────┬─────────┬─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬────────────────────────────────────┬───────────────────────────────┬───────────────────┬─────────┬───────────────────────────────────────────────────────────────┬────────────────────────────────────────────────────────────┬─────────────────────────────────────────────────┬────────────────────────┬────────────────────────────┬───────────────────────┬────────────────────────┬──────────────────────────────┬─────────────┬─────────────────────────────┬────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┬─────────────────────────┬───────────────────┬─────────────────────────────────┬─────────────────────────────────────────────────────────────

In [27]:
con.close()